##  Profit & Loss Statement Extractor Model Package from AWS Marketplace 


This solution takes an annual report in a digital PDF format as input, and returns the JSON data corresponding to the extracted tabular data from the report's Profit & Loss statements. The solution utilizes computer vision techniques to identify the pages with tables, as well as text-based classification techniques to determine the relevance of the pages in question. This is done extremely quickly, rendering it unnecessary for an individual to go through hundreds of pages before finding the required information.

The Profit & Loss Data Extractor uses text classification techniques to identify the pages containing information about Profit & Loss. It then utilizes table detection and extraction algorithms to narrow the search to only those pages that contain tabular data, before finally retrieving it.

This solution can be used in industries like consulting, banking, financial services, insurance, retail, healthcare, pharmaceuticals, manufacturing, airlines, etc to automate processes  like financial spreading, vendor/merchant risk assessment, fundamental analysis etc.

This sample notebook shows you how to deploy Mphasis DeepInsights:  Profit & Loss Statement Extractor.



#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis DeepInsights:  Profit & Loss Statement Extractor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page ** Profit & Loss Statement Extractor.**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [3]:
model_package_arn = 'arn:aws:sagemaker:us-east-2:786796469737:model-package/profit-and-loss-statement-extractor'

In [4]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
#from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
#import urllib.request
import numpy as np

In [5]:
content_type='application/zip'
model_name='ar-profit-loss-model-v1'
real_time_inference_instance_type='ml.m5.4xlarge'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [6]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

#### A. Create an endpoint

In [7]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

----!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

Input.zip

	|--Input
    
		|--sample_financial_report.zip

`file_to_upload.zip`

In [8]:
file_name ='Input.zip'

#### C. Perform real-time inference

In [9]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'ar-profit-loss-model-v1'\
    --body fileb://$file_name \
    --content-type 'application/zip' \
    --region us-east-2 \
    output1.zip

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


<Add code snippet that shows the payload contents>

#### D. Visualize output

Unzip the output.zip file. That will contain the extracted " Profit & Loss" csv files

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
predictor.delete_endpoint()

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [11]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.xlarge')
transformer.transform("s3://mphasis-marketplace/financial_table_profit_loss/input/Input.zip", content_type='application/zip')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")
bucketFolder = transformer.output_path.rsplit('/')[3]

...................... * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 128-969-187
 * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 128-969-187
169.254.255.130 - - [16/Nov/2021 14:41:11] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [16/Nov/2021 14:41:11] "#033GET /execution-parameters HTTP/1.1#033" 404 -
/opt/program/image_module/Input/mphasis-annual-report-2021.pdf
169.254.255.130 - - [16/Nov/2021 14:41:11] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [16/Nov/2021 14:41:11] "#033GET /execution-parameters HTTP/

In [12]:
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output.zip', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/Input.zip.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


## Output will be inside a zip file

### 4. Clean-up

#### A. Delete the model

delete the endpoint after you have used it to save resources.


In [13]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

